In [9]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config import gkey

In [2]:
counties_df = pd.read_csv("Poverty_Data/2017_Poverty_Data.csv")

In [3]:
counties_df['lat'] = ""
counties_df['lng'] = ""

In [4]:
params = {"key":gkey}
for index, row in counties_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    county_type = row['County']
    state_type = row['State']
    params['address'] = f"{county_type},{state_type}"
    county_lat_lng = requests.get(base_url, params=params).json()
    counties_df.loc[index, "lat"] = county_lat_lng["results"][0]["geometry"]["location"]["lat"]
    counties_df.loc[index, "lng"] = county_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [5]:
gmaps.configure(api_key=gkey)

In [6]:
locations = counties_df[["lat", "lng"]].astype(float)

poverty_rate = counties_df["Total Poverty Rate (%)"].astype(float)

In [11]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 90
heat_layer.point_radius = 1

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))